In [ ]:
from bokeh.plotting import figure
from bokeh.layouts import row, column, widgetbox, gridplot

from bokeh.io import curdoc
from bokeh.io import output_notebook, show, output_file

from bokeh.models import ColumnDataSource, HoverTool, TapTool, Range1d, OpenURL
from bokeh.models import LinearColorMapper , ColorBar
from bokeh.models.widgets import Select, Slider


import numpy as np

In [ ]:
# =============================================
# THIS comes from INTERFACE
#
selected_exposure = '00000003'# args['exposure']
selected_arm = 'r' # args['arm']
selected_spectrograph =  4 # args['spectrograph']

# =============================================
# THIS comes from QLF.CFG
#
night = '20190101'

# ============================================
#  THIS READ yaml files
#
from scalar_metrics_v02 import LoadMetrics

cam = selected_arm+str(selected_spectrograph)
exp = selected_exposure # intentionaly redundant
lm = LoadMetrics(cam, exp, night);
metrics, tests  = lm.metrics, lm.tests 

# =============================================
# THIS is only to simplify the code understanding
#
countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

In [11]:
hist_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Frequency: </span>
            <span style="font-size: 13px; color: #515151">@hist</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">sigma: </span>
            <span style="font-size: 13px; color: #515151;">[@left, @right]</span>
        </div>
    </div>
"""
sname = 'XSIGMA'

def bins_doane(data):
    # Dane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in xwsigma[sname]])
    b = b/sum([ (x - mean)**2 for x in xwsigma[sname]])**(1.5)
    return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))

bins = bins_doane(xwsigma[sname]) # formely: 17
hover = HoverTool(tooltips=hist_tooltip)


hist, edges = np.histogram(xwsigma[sname], bins = bins)

source = ColumnDataSource(data={
    'hist':hist,
    'bottom':[0] *len(hist),
    'left':edges[:-1],
    'right':edges[1:]
})

p = figure(title=sname,tools=[hover,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label=sname,
           #y_axis_type='log',
            background_fill_color="white")

p.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source, 
        fill_color="dodgerblue", line_color="black", alpha=0.8,
       hover_fill_color='blue', hover_line_color='black', hover_alpha=0.8)
p.legend.location = "top_left"
output_notebook()
show(p)

Loading BokehJS ...

## Outro

In [ ]:
def bins_doane(data):
    # Doane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in xwsigma['XSIGMA']])
    b = b/sum([ (x - mean)**2 for x in xwsigma['XSIGMA']])**(1.5)
    return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
    
ndata = len(xwsigma['XSIGMA'])
mean = np.mean(xwsigma['XSIGMA'])
sigma = np.std(xwsigma['XSIGMA'])

b = sum([ (x - mean)**3 for x in xwsigma['XSIGMA']])
b = b/sum([ (x - mean)**2 for x in xwsigma['XSIGMA']])**(1.5)
danesrule= int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
sturgesrule = int(round(1.+3.322 *np.log10(500) ))
scottsrule= int(round(3.49 *sigma/(ndata)**(1./3.)))

In [ ]:
1 + 3.322 *np.log10(55) #= 6.781 ≅ 7

In [ ]:
print(danesrule, sturgesrule, scottsrule)